In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,6"

os.environ['CUDA_PATH']='/usr/local/cuda-11'

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
import re

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
tokenizer.add_special_tokens({'pad_token': '<?>'})

0

In [5]:
from accelerate import init_empty_weights, infer_auto_device_map, load_checkpoint_and_dispatch
from transformers import AutoConfig, AutoModelForCausalLM

In [6]:
config = AutoConfig.from_pretrained('tiiuae/falcon-7b', trust_remote_code=True)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
model.tie_weights()

device_map = infer_auto_device_map(model, max_memory={0: "5GiB",1: "5GiB",2: "5GiB",3: "5GiB",4: "5GiB",5: "5GiB"}, 
                                   no_split_module_classes=["DecoderLayer"])

model = load_checkpoint_and_dispatch(model, 
                                     "/mnt/data/mart/test_trainer/checkpoint-126/", 
                                     device_map=device_map)

model.hf_device_map

{'transformer.word_embeddings': 0,
 'lm_head': 0,
 'transformer.h.0': 0,
 'transformer.h.1': 0,
 'transformer.h.2': 0,
 'transformer.h.3': 0,
 'transformer.h.4': 1,
 'transformer.h.5': 1,
 'transformer.h.6': 1,
 'transformer.h.7': 1,
 'transformer.h.8': 1,
 'transformer.h.9': 1,
 'transformer.h.10': 2,
 'transformer.h.11': 2,
 'transformer.h.12': 2,
 'transformer.h.13': 2,
 'transformer.h.14': 2,
 'transformer.h.15': 2,
 'transformer.h.16': 3,
 'transformer.h.17': 3,
 'transformer.h.18': 3,
 'transformer.h.19': 3,
 'transformer.h.20': 3,
 'transformer.h.21': 3,
 'transformer.h.22': 4,
 'transformer.h.23': 4,
 'transformer.h.24': 4,
 'transformer.h.25': 4,
 'transformer.h.26': 4,
 'transformer.h.27': 4,
 'transformer.h.28': 5,
 'transformer.h.29': 5,
 'transformer.h.30': 5,
 'transformer.h.31': 5,
 'transformer.ln_f': 5}

In [7]:
model = model.eval()

In [8]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

In [9]:
sequences = pipeline(
   "The user is interested in a tool to find relationships between medical terms.",
    max_length=1024,
    do_sample=True,
    top_k=1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: The user is interested in a tool to find relationships between medical terms.<___:::('('-('-('----------------
















:::::ersenceenceersenceersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersersers.---------.-.-----.-.-.......................................................
.....
.....
..




























































  - - - - -  - -  - -    - - - -      
